# Qualtrics integration and pipeline
Demonstration of how we will generate archetypes from the *Qualtrics* responses both for the current car as well as for each vintage (i.e. archetype defined by `vehicle_type` and `fuel_type`)

## Qualtrics

In [ ]:
import os

from QualtricsAPI.Setup import Credentials

from QualtricsAPI import Responses

In [ ]:
TOKEN = os.environ.get('QUALTRICS_API_KEY')
DATA_CENTER = os.environ.get('QUALTRICS_BASE_URL').split('.')[0]

In [ ]:
Credentials().qualtrics_api_credentials(token=TOKEN, data_center=DATA_CENTER)

In [ ]:
r = Responses()
survey_id = 'SV_6u4YzY24OAfhWfQ'
df_pre_study = r.get_survey_responses('SV_6u4YzY24OAfhWfQ')

In [ ]:
df_pre_study.head()

## Workflow

In [ ]:
import tcsscraper.scrape as tcs

from tcsscraper.helper import Car

In [ ]:
Car.vehicle_classes

In [ ]:
Car.fuel_types

In [ ]:
# car = Car('Luxusklasse', 'Elektro', 5)

In [ ]:
car = Car('Luxusklasse', 'Benzin', 10)
print(car)

In [ ]:
similar_cars = tcs.get_similar_cars(car, km=20e3, canton='ZH', buffer=5, verbose=True)

In [ ]:
len(similar_cars)

In [ ]:
def remove_unit(string):
    """Removes unit from string like 17'000 CHF/year -> float(17e3)

    Args:
        string (str): string with unit
    """
    number, unit = string.split(' ')
    number = number.replace("'", '')
    return float(number)

In [ ]:
import pandas as pd

def generate_archetype(similar_cars, ndigits=2):
    """Generates an archetypical car from a list of similar cars

    Args:
        similar_cars (list of dict): returned by get_similar_cars()
        ndigits (int): round average values to
    """
    car_attributes = []
    for c in similar_cars:
        costs = c['costs']
        attrs = {
            'fix_cost': remove_unit(costs['Fixe Kosten']),
            'variable_cost': remove_unit(costs['Variable Kosten']),
            'cost_per_km': remove_unit(costs['Kilometerkosten'])
        }
        car_attributes.append(attrs)

    car_attributes = pd.DataFrame(car_attributes)

    archetype = dict(round(car_attributes.mean(), ndigits=ndigits))

    return archetype
    

In [ ]:
archetype = generate_archetype(similar_cars)
archetype

## Pipeline

### User specific archetype

In [ ]:
## generate data.frame with variables to feed to get_similar_cars
df_qualt = pd.DataFrame({
    'vehicle_type': ['SUV S', 'Luxusklasse', 'Mittelklasse'],
    'fuel_type': ['Elektro', 'Benzin', 'Benzin'],
    'fuel_consumption': [14, 7, 2],  ## if electric => kWh/100km
    'km': [15e3, 20e3, 22e3],
    'canton': ['VD', 'AI', 'ZG']
})

df_qualt.head()

In [ ]:
def helper(row, max_buffer=5, verbose=True):
    car = Car(row['vehicle_type'], row['fuel_type'], row['fuel_consumption'])
    init = 1
    if verbose:
        print('---\n{}\n---'.format(car))
    while(init < max_buffer+1):
        try:
            sc = tcs.get_similar_cars(car, row['km'], row['canton'], buffer=init, verbose=verbose)
        except Exception:
            init += 1
            if verbose:
                print('increaseing buffer to {}'.format(init))
            continue
        return sc
    return None

In [ ]:

df_qualt['similar_cars'] = df_qualt.apply(lambda row: helper(row), axis=1)

In [ ]:
## generate archetypes
df_qualt['archetype'] = df_qualt.apply(lambda row: generate_archetype(row['similar_cars']), axis=1)
df_qualt['archetype']

In [ ]:
## unpack stuff
df_qualt = df_qualt.assign(
    archetype_fix_cost=lambda x: [y["fix_cost"] for y in x["archetype"]],
    archetype_variable_cost=lambda x: [y["variable_cost"] for y in x["archetype"]],
    archetype_cost_per_km=lambda x: [y["cost_per_km"] for y in x["archetype"]]
)


### Generic archetypes

In [64]:
df_base = tcs.get_base_table()

## Remarks
- `fuel_consumption` with `fuel_type='electric'` see **Normverbrauch**: https://www.verbrauchskatalog.ch/index.php => higher values and unit kWh/100km